In [1]:
import numpy as np

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x*y
    def backward(self, dout):
        dx = self.y*dout
        dy = self.x*dout
        return dx, dy

In [3]:
apple_price = 100
apple_count = 2
tax_rate = 1.1

apple_layer = MulLayer()
tax_layer = MulLayer()

sum_price = apple_layer.forward(apple_price, apple_count)
total_price = tax_layer.forward(sum_price, tax_rate)
print(f'Total Price: {total_price}')

dsum_price, dtax_rate = tax_layer.backward(1)
print(f'Differential of sum_price: {dsum_price}')
print(f'Differential of dtax_rate: {dtax_rate}')
dapple_price, dapple_count = apple_layer.backward(dsum_price)
print(f'Differential of apple_price: {dapple_price}')
print(f'Differential of apple_count: {dapple_count}')

Total Price: 220.00000000000003
Differential of sum_price: 1.1
Differential of dtax_rate: 200
Differential of apple_price: 2.2
Differential of apple_count: 110.00000000000001


In [4]:
# 덧셈 레이어
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x+y
        return out
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

In [6]:
# Relu 활성화 함수 레이어
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = x <= 0
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        # dout 는 배열
        # if dout > 0:
        #     return dout
        # return 0

        # 미분 값이 1이라는 것은 입력 받은것을 그대로 전달한다는 것을 의미, 덧셈 레이어도 입력값이 1을 곱해 그대로 전달해줌
        # 렐루에서 입력 값이 0보다 크면 미분값이 1, 작거나 같으면 0 즉 0보다 크면 값을 그대로 전달
        dx = dout[self.mask] = 0
        return dx


In [1]:
# Sigmoid 활성화 함수 레이어
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        y = 1/(1+np.exp(-x))
        self.out = y
        return y
    def backward(self, dout):
        out = dout*self.out(1.0-self.out)
        return out


In [ ]:
# Affine(행렬의 내적)
class Affine:
    def __init__(self, W, B):
        self.W = W
        self.B = B
        self.dW = None # 가중치의 미분(기울기)        
        self.dB = None # 편향의 미분
    def forward(self, X, W, B):
        self.X = X
        self.W = W
        self.B = B
        return np.dot(X,W)+B
    def backward(self, dout):
        # 예 X(2,3), W(3,2), B(2) 형태의 경우
        # 순전파로 결과가(2,2)가됨
        # 역전파의 경우
        # 전달값(L)이 (2,2)임으로 편향(2) 형태로 저장하기 위해서는 sum(axis=0) 수행해야함
        # X의 미분(dX)이 (2,3) 형태를 유지하기 위해서는 (L(2,2) dot W.T(2,3)) 순서로 내적해야함
        # W의 미분(dW)이 (3,2) 현태를 유지하기 위해서는 (X.T(3,2) dot L(2,2)) 순서로 내적해야함
        self.dB = np.sum(dout, axis=0)
        dX = np.dot(dout, self.W.T)
        self.dW = np.dot(dX, dout)
        return dX